In [1]:
import pandas as pd
import os
import glob
import re
import json

In [2]:
def write_json(data_dict,filename,data_write_path):
    '''
    Write JSON file into designated raw data folder.
    '''
    
    # Serializing json
    json_object = json.dumps(data_dict, indent=4)

    # Writing to .json
    with open(data_write_path + filename, "w") as outfile:
        outfile.write(json_object)

In [3]:
combined_df = pd.read_csv('./data/cleaned/combined/combined_results_parliament.csv', converters = {'parliament_code_digits': str})
combined_df.fillna('',inplace=True)

In [4]:
combined_df_year_district = combined_df.set_index(['year','parliament_code_digits'])

In [5]:
year_district_pairs = combined_df_year_district.index.unique().to_list()

In [6]:
nested_year_district = {}

In [7]:
for year_district in year_district_pairs:

  [year,district] = year_district
  
  if year not in nested_year_district:
    nested_year_district[year] = {}
  
  nested_year_district[year][district] = {}

  district_columns = ['total_votes','constituency','state']

  result_columns = ['name','party_code','coalition','votes','vote_share','winner']

  nested_year_district[year][district]['info'] = combined_df_year_district.loc[(year,district),:][district_columns].to_dict('records')[0]

  nested_year_district[year][district]['results'] = combined_df_year_district.loc[(year,district),:][result_columns].to_dict('records')

  winner = [row for row in nested_year_district[year][district]['results'] if row['winner'] == 1][0]

  nested_year_district[year][district]['winner'] = winner

    

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1761: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [8]:
nested_year_district[2004]['002']

{'info': {'total_votes': 32448, 'constituency': 'KANGAR', 'state': 'Perlis'},
 'results': [{'name': 'DATUK RADZI SHEIKH AHMAD',
   'party_code': 'UMNO',
   'coalition': 'BN',
   'votes': 22498,
   'vote_share': 69.33555226824457,
   'winner': 1},
  {'name': 'ISHAR SAAD',
   'party_code': 'PAS',
   'coalition': 'BA',
   'votes': 9950,
   'vote_share': 30.664447731755423,
   'winner': 0}],
 'winner': {'name': 'DATUK RADZI SHEIKH AHMAD',
  'party_code': 'UMNO',
  'coalition': 'BN',
  'votes': 22498,
  'vote_share': 69.33555226824457,
  'winner': 1}}

In [9]:
write_json(nested_year_district,'nested_results_parliament.json','./data/cleaned/combined/')